# Timer

In [3]:
from time import sleep
from datetime import datetime as dt

start_time = "13.01.2023, 17:25"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [5]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'sell'
pattern = ['STOCH', 'RSI', 'LinearReg']
indicator_list = pattern
indicator_list_higher = ['LinearReg']

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 1000
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = indicator_list
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [25]},
                'LinearReg': {'timeperiod': [2, 3, 4, 5, 6, 8], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 15m/1h, trade type is sell
Number of combinations is 6


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:22<00:00,  3.67s/it]


# Check local statistics

In [4]:
# 15m/1h
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']

pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('e_ratio_rank', ascending=False).head(20)
stat

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
0,STOCH_RSI_LinearReg,14,35,9,7,3,25,2,0,81,1.778096,-0.621667,63.025763,-50.355000,1.5063,1.7681,2.3062,2.3102,2.5375,2.3372,2.0977,2.0980,1.8991,1.8355,1.7998,1.7215,1.6719,1.6054,1.5779,1.5306,1.4960,1.4580,1.4971,1.4653,1.5163,1.5159,1.5341,1.5887,-0.33,-0.43,-0.80,-0.71,-0.87,-0.79,-0.79,-0.87,-0.63,-0.70,-0.56,-0.46,-0.49,-0.41,-0.45,-0.46,-0.39,-0.54,-0.56,-0.44,-0.75,-0.84,-0.78,-0.87
1,STOCH_RSI_LinearReg,14,35,9,7,3,25,3,0,38,1.754842,-0.541250,28.683983,-20.567500,1.2928,1.6617,2.4453,2.2211,2.5354,2.1721,2.0348,2.1008,1.8676,1.8332,1.7931,1.6689,1.6424,1.5345,1.4977,1.4402,1.4320,1.4225,1.4808,1.4687,1.5330,1.6261,1.6446,1.7669,-0.31,-0.34,-0.83,-0.70,-0.80,-0.61,-0.45,-0.68,-0.48,-0.58,-0.39,-0.35,-0.15,-0.31,-0.15,-0.12,-0.31,-0.35,-0.54,-0.42,-0.82,-0.96,-0.91,-1.43
2,STOCH_RSI_LinearReg,14,35,9,7,3,25,4,0,25,1.611492,-0.415000,15.287292,-10.375000,1.3087,1.7393,2.3717,1.9085,2.1453,1.7866,1.6421,1.7417,1.7287,1.6861,1.6424,1.5642,1.5397,1.4631,1.4173,1.3254,1.3177,1.3071,1.3729,1.3876,1.4472,1.5484,1.5639,1.7202,-0.27,-0.27,-0.80,-0.69,-0.54,-0.53,-0.17,-0.22,-0.25,-0.29,-0.33,-0.25,-0.08,-0.25,-0.13,-0.08,-0.07,-0.25,-0.09,-0.17,-0.75,-0.91,-0.83,-1.74
4,STOCH_RSI_LinearReg,14,35,9,7,3,25,6,0,21,1.411996,-0.355000,8.651912,-7.455000,0.8900,1.5589,2.0255,1.5923,1.7160,1.4690,1.3611,1.4795,1.4953,1.5047,1.4866,1.4162,1.3857,1.3035,1.2825,1.2667,1.2389,1.2114,1.2847,1.2744,1.3252,1.3806,1.4076,1.5316,-0.17,-0.27,-0.62,-0.64,-0.73,-0.53,-0.27,-0.21,-0.28,-0.70,-0.41,-0.41,-0.15,-0.25,-0.15,-0.18,-0.07,-0.19,-0.12,-0.13,-0.21,-0.62,-0.44,-0.77
3,STOCH_RSI_LinearReg,14,35,9,7,3,25,5,0,21,1.358162,-0.390000,7.521412,-8.190000,1.0174,1.5083,2.0131,1.5612,1.6919,1.4447,1.3254,1.4296,1.4323,1.4159,1.3856,1.2986,1.2767,1.2227,1.2042,1.1222,1.1104,1.0966,1.1906,1.2094,1.2745,1.3863,1.4037,1.5746,-0.23,-0.12,-0.62,-0.69,-0.50,-0.33,-0.15,-0.21,-0.25,-0.29,-0.33,-0.25,-0.08,-0.25,-0.08,-0.08,-0.07,-0.21,-0.07,-0.17,-0.75,-0.98,-0.91,-1.74
5,STOCH_RSI_LinearReg,14,35,9,7,3,25,8,0,22,1.329058,-0.407917,7.239283,-8.974167,0.9665,1.4780,1.6145,1.3312,1.4584,1.3064,1.2415,1.3750,1.4342,1.4222,1.4119,1.3498,1.3323,1.3002,1.2998,1.2116,1.1932,1.1836,1.2468,1.2472,1.2970,1.3461,1.3775,1.4725,-0.17,-0.31,-0.46,-0.43,-0.57,-0.54,-0.35,-0.36,-0.64,-0.75,-0.54,-0.49,-0.38,-0.42,-0.46,-0.22,-0.05,-0.20,-0.34,-0.25,-0.16,-0.45,-0.54,-0.71


# Research parameter influence

In [81]:
param = 'LinearReg_timeperiod'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,LinearReg_timeperiod,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,2,1.937883,-0.562604,65.569898,-39.616042,247
1,3,2.747457,-1.018333,76.114432,-44.385000,123
2,4,4.568197,-2.099375,56.411997,-33.292188,91
3,5,3.548132,-1.371806,74.680826,-39.263194,80
4,6,3.757032,-1.289444,80.305944,-37.298889,80
5,8,3.517467,-1.098056,77.010065,-33.492500,86


# Plot result in file

In [77]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = 'LinearReg_timeperiod', 'pct_price_diff_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig('temp.png', dpi=fig.dpi)

# Save new config data to config file

In [18]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'RSI': {'timeperiod': [14], 'low_bound': [35]},
                'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                          'slowd_period': [3], 'low_bound': [25]},
                'LinearReg': {'timeperiod': [3], 'low_bound': [0]}
              }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [3]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            9                  7                  3                  30                            83.541667   
                                                                                      25                            85.785833   
12             25            9                  7                  3                  30                            88.782083   
14             30            9                  7                  3                  25                            80.833333   
12             25            9                  7                  3                  25                            87.847917   
14             25            9                  7                  3                  30                            89.395000   
                                                                                      20                            89.395000   
                                                                                      25                            89.395000   
12             25            9                  7                  3                  20                            89.395000   
16             30            9                  7                  3                  20                            84.820833   
14             30            9                  7                  3                  30                            78.804583   
16             25            9                  7                  3                  20                            88.750000   
                                                                                      25                            88.750000   
                                                                                      30                            88.750000   
12             30            9                  7                  3                  30                            75.416667   
14             30            9                  7                  3                  20                            79.687500   
16             15            5                  4                  3                  25                            93.229167   
12             25            9                  7                  3                  15                            91.664583   
14             25            9                  7                  3                  15                            91.664583   
16             15            7                  4                  5                  25                            95.025694   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            9                  7                  3                  30                        -1.760000   
                                                                                      25                        -1.970833   
12             25            9                  7                  3                  30                        -2.084167   
14             30            9                  7                  3                  25                        -1.777917   
12             25            9                  7                  3                  25                        -2.421250   
14             25            9                  7                  3                  30                        -2.759583   
                                                                                      20                        -2.759583   
                                                                                      25                        -2.759583   
12            